<a href="https://colab.research.google.com/github/Victor1992/nlp_tensorflow_disaster_detection/blob/main/introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in tensorflow

NLP has the goal of deriving information out of natural language( could be sequences text or speech)

Another common term for NLP is sequence to sequence problem (seq2seq).

## Check for GPU

In [2]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-49046f02-cb82-102c-a7cd-e3a74d8ed806)


## Get helper functions


In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys


--2021-12-18 08:47:47--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-12-18 08:47:47 (37.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get text dataset

The dataset we are going to use is kaggle's dataset fro introduction to NLP dataset (text samples of Tweets labelled as disaster ot not disaster) : https://www.kaggle.com/c/nlp-getting-started


In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data('nlp_getting_started.zip')

--2021-12-18 08:47:53--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.02s   

2021-12-18 08:47:53 (38.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize a text dataset

In [5]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [6]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle  training data
train_df_shuffled = train_df.sample(frac=1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# How test data looks like ?
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

## Splitting data into training and testing dataset


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                             train_df_shuffled["target"].to_numpy(),
                                                                             test_size=0.1, # use 10% of data for random split
                                                                             random_state=42)

In [13]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [14]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default text vectorization parameters

text_vectorizer = TextVectorization(max_tokens=10000, # how many words are there in vocab
                  standardize="lower_and_strip_punctuation",
                  split="whitespace",
                  ngrams=None, # Creating group of n words
                  output_mode="int", # how to map tokens to numbers
                  output_sequence_length=15, # how long do u wnat ypur sequences to be
                  pad_to_max_tokens = True
                  )

In [16]:
# Fit the vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "Hi i am varun sharma"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1663,    8,  160,    1,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [18]:
# choose a random senetcnce form train sentences and tokenize it
import random
random_sentence = random.choice(train_sentences)
print(f"Original tweet:\n {random_sentence}\
        \n\n Vectorized version:")
text_vectorizer([random_sentence])

Original tweet:
 @DrMartyFox     In the U.S. government and Libs made evil good and good evil. We will reap the whirlwind. Lord have mercy on us.        

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    4,    2,   69,  547,    7,    1,  299, 2121,  136,    7,
         136, 2121,   46,   38]])>

In [19]:
# Get the unique words from vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all unique words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in Vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")



Number of words in Vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating an embedding using embedding layer

An embedding is a rich numerical representation of words which can be learned during training

To make our embedding, we are going to use Tensotflow embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care about most for our embedding layer:
* `input_dim` = the size of our vacabulary
* `output_dim` =  the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [20]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = 10000,
                             output_dim = 128, # output shape
                             input_length = 15)
embedding

In [21]:
# Get a random sentence from atrainig set
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\
        \n\n Embedded Version:")

# Embed the random sentence (turn it into a dense vectors if fixed size)
embedding(text_vectorizer([random_sentence]))

Original text:
 BLOG: Rain much needed as drought conditions worsen: Right now Charlotte and much of the surrounding area haveÛ_ http://t.co/OLzaVTJFKH        

 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01041617, -0.03792292, -0.01362002, ..., -0.0133626 ,
         -0.013388  ,  0.04361517],
        [-0.04102676, -0.01201153, -0.02892603, ...,  0.01181883,
          0.03025654,  0.03210194],
        [-0.04422691, -0.03274629, -0.02044604, ..., -0.04906574,
          0.0141464 , -0.0213517 ],
        ...,
        [-0.04422691, -0.03274629, -0.02044604, ..., -0.04906574,
          0.0141464 , -0.0213517 ],
        [-0.0414507 , -0.02506554,  0.01537037, ..., -0.04879241,
         -0.01123076,  0.04102829],
        [-0.00352373,  0.00878451, -0.03583599, ..., -0.01205517,
         -0.03740895, -0.00545676]]], dtype=float32)>

from sklearn import model_selection
## Modelling a test dataset( runnign a seroes of experiments)

Now as we have turned our text sequences inot numbers, it's time to start building a series of modelling experiements.

We'll start with a baseline and move on from there.

* Model 0 : Naive Bayes
* Model 1 : Feed forward neural network(dense model)
* Model 2 : LSTM Model (RNN)
* Model 3 : GRU Model (RNN)
* Model 4 : Bidirectional-LSTM model_selection
* Model 5 : 1D CNN
* Model 6 : tensorflow Hub Pretrained Feature Extractor(using transfer learning for NLP)

Standear Steps in modelling with tensorflow

* Create a  model
* Build a model
* Fit the model
* evaluate a model

### Model 0: Getting a baseline

As with all ML modelling experiments, it's importnant to craete a baseline model so ypu'he got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn Multinomial Naive Bayes using the TF_IDF formula to convert our words to numbers

**Note:** Its common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see we can improve upon

#### Create Baseline Model (Naive Bayes)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create Tokenization and modelling Pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # Convert words into numbers
                    ("clf", MultinomialNB()) # Model the text 
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

#### Evaluate the model

In [23]:
# evaluate the model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our Baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our Baseline model achieves an accuracy of: 79.27%


#### Make Predictions

In [24]:
# Make Predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

#### Let's evaluate on different Metrics

* Accuracy
* Precision
* Recall
* F1-Score

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuarcy, precision, recall and f1 score of a binar calssification model.
  """
  # calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall, and f1-score using "weighted avg"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {
      "accuracy" : model_accuracy,
      "precision": model_precision,
      "model_recall" : model_recall,
      "f1": model_f1
  }
  return model_results

In [26]:
# Get Baseline results
baseline_results = calculate_results(y_true=val_labels, y_pred = baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'model_recall': 0.7926509186351706,
 'precision': 0.8111390004213173}

### Model 1. A simple dense layer 

In [27]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboward logs
SAVE_DIR = "model_logs"

In [28]:
# build model with the functional api
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x =text_vectorizer(inputs)
x =embedding(x)
x= layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [29]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

#### Compile the Model

In [30]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

#### Fit the Model

In [31]:
# Fit the model
model_1_history = model_1.fit(x= train_sentences,
                              y=train_labels,
                              epochs =5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20211218-084801
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.6138 - accuracy: 0.6916 - val_loss: 0.5377 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.4444 - accuracy: 0.8189 - val_loss: 0.4689 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3489 - accuracy: 0.8594 - val_loss: 0.4561 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2867 - accuracy: 0.8907 - val_loss: 0.4634 - val_accuracy: 0.7979
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2400 - accuracy: 0.9136 - val_loss: 0.4776 - val_accuracy: 0.7874


#### Evaluate the Model

In [32]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4776 - accuracy: 0.7874


[0.4776366949081421, 0.787401556968689]

#### Make Predictions

In [33]:
model_1_preds_probs = model_1.predict(val_sentences)
model_1_preds_probs[10]


array([0.11226848], dtype=float32)

In [34]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [35]:
# Convert model preictions probablities to label format
model_1_preds = tf.squeeze(tf.round(model_1_preds_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

#### Evalute the model

In [36]:
# Calculate model 1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred = model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'f1': 0.7842639469124086,
 'model_recall': 0.7874015748031497,
 'precision': 0.7927656871284042}

In [37]:
import numpy as np
np.array(list(model_1_results.values()))  > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Model 2: Recurrent neural network (RNN)

The premise of RNN is to use the representation of previous input to aid the representation of later input

LSTM: lomg short term memory

Structure of RNN will look like

```
Input(text) --> Tokenize --> Embedding --> LSTM(RNNs/ dense) --> Output(label Probablity)
```

**Resources**

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/



#### Create LSTM Model

In [38]:
from keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x =text_vectorizer(inputs)
x = embedding(x)
x =layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [39]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

#### Compile the Model

In [40]:
model_2.compile(loss ="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


#### Fit the model

In [41]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs =5,
                              validation_data=(val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_2_LSTM") ])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20211218-084812
Epoch 1/5
215/215 [==============================] - 7s 13ms/step - loss: 0.2265 - accuracy: 0.9158 - val_loss: 0.5521 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1596 - accuracy: 0.9416 - val_loss: 0.5941 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1263 - accuracy: 0.9508 - val_loss: 0.6954 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1038 - accuracy: 0.9610 - val_loss: 0.8122 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0842 - accuracy: 0.9695 - val_loss: 0.8201 - val_accuracy: 0.7703


#### Make Predcitions

In [42]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:5]

array([[0.02351562],
       [0.87275714],
       [0.9997706 ],
       [0.07535937],
       [0.00119803]], dtype=float32)

In [43]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

#### Evaluate Model 2 

In [44]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.03412073490814,
 'f1': 0.7679241301282296,
 'model_recall': 0.7703412073490814,
 'precision': 0.7726339681635592}

In [45]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'model_recall': 0.7926509186351706,
 'precision': 0.8111390004213173}

In [46]:
np.array(list(model_2_results.values())) > np.array(list(model_2_results.values()))

array([False, False, False, False])

### Model 3: GRU

**Resourse**

https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be

#### Create GRU model

In [47]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x =embedding(x)
x =layers.GRU(64)(x)
# print(x.shape)
# x =layers.GRU(64, return_sequences =True)(x)
# print(x.shape)
# x =layers.LSTM(42, return_sequences= True)(x)
# print(x.shape)
# x =layers.GRU(99)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [48]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

#### Compile the model


In [49]:
model_3.compile(loss= "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"]) 

#### Fit the model

In [ ]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks =[create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20211218-084836
Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.1609 - accuracy: 0.9369 - val_loss: 0.9024 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0844 - accuracy: 0.9680 - val_loss: 0.8887 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0704 - accuracy: 0.9730 - val_loss: 0.9756 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0631 - accuracy: 0.9743 - val_loss: 0.9492 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0527 - accuracy: 0.9777 - val_loss: 1.3095 - val_accuracy: 0.7743


#### Make Predictions

In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

In [ ]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

#### Evaluate model 3


In [ ]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred = model_3_preds)
model_3_results

In [ ]:
np.array(list(model_3_results.values())) > np.array(list(baseline_results.values()))

### Model 4 Bidirectional RNN

#### Create a Bidirectional RNN Model 

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional") 

In [ ]:
model_4.summary()

#### Compile the model

In [ ]:
model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

#### Fit the model

In [ ]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_4_bidirectional")])

#### Make Predictions

In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)

In [ ]:
# Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

#### Calculate the results

In [ ]:
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred = model_4_preds)
model_4_results

### Model 5 Convolutional Neural Network for Text

Typical structure of a Conv1D model for sequences(text)

```
Inputs(text)--> Tokenization--> Embedding-->Layers(typically ConvD+ Pooling) -> ouputs (class probablities)
```

In [ ]:
# Test our embedding , Conv1d and max pooling
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        strides=1,
                        activation="relu", # relu = id value is -ve it converst to 0, and if +ve it remains the same
                        padding="valid")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape